# 智能RSI参数

根据个股一年来最低的RSI来判断当前股价是否处于最低位置

In [16]:
from alpha.notebook import *
import traceback

from alpha.core.rsi_stats import RsiStats, rsiday, rsi30
await init_notebook()

def qualified(code):
    if "." not in code:
        if code.startswith("6"):
            code += ".XSHG"
        else:
            code += ".XSHE"
        
    return code

def day(code, value):
    qualified(code)
    return round(rsiday.get_proba(code, value), 2)

def m30(code, value):
    code = qualified(code)
        
    return round(rsi30.get_proba(code, value), 2)

In [ ]:
# rsi = RsiStats(FrameType.MIN30)
# await rsi.calc("2021-09-30 15:00")

In [ ]:
# rsiday = RsiStats(FrameType.DAY)
# await rsiday.calc("2021-09-30")

In [3]:
async def check_rsi(names, tm=None):
    codes = [name_to_code(name) for name in names]
        
    results = []
    for code in codes:
        sec = Security(code)
        name = sec.display_name
        
        ft = FrameType.MIN30
        end = arrow.get(tm) if tm else arrow.now()
        if end.hour == 0:
            end = tf.combine_time(end, hour = 15)
            
        start = tf.shift(tf.floor(end,ft), -39, ft)
        bars = await sec.load_bars(start, end, ft)
        bars = bars[np.isfinite(bars["close"])]
        
        r30 = relative_strength_index(bars["close"])[-3:]
        r30 = np.round(r30, 2)
        prsi30 = [rsi30.get_proba(code, r) for r in r30]
        
        ft = FrameType.DAY
        end = end.date()
        start = tf.day_shift(end, -39)
        bars  = await sec.load_bars(start, end, ft)
        bars = bars[np.isfinite(bars["close"])]
        
        rday = relative_strength_index(bars["close"])[-3:]
        rday = np.round(rday, 2)
        prday = [rsiday.get_proba(code, r) for r in rday]
        
        flag = ""
            
        try:
            if min(prsi30) < 0.1:
                print(f"{name}\t^\t{prsi30}\t30分钟")
            if min(prday) < 0.1:
                print(f"{name}\t^^\t{prday}\t日线")
                
            if max(prsi30) >= 0.9:
                print(f"{name}\tv\t{prsi30}\t30分钟")
            if max(prday) >= 0.9:
                print(f"{name}\tvv\t{prday}\t日线")

            if min(prsi30) < 0.1:
                flag += "^" 
            elif max(prsi30) >= 0.9:
                flag += "v"

            if min(prday) < 0.1:
                flag = " | ^^" 
            elif max(prday) >= 0.9:
                flag = " | vv"
        except Exception:
            continue
            
        results.append([name, code, prday[-1], prsi30[-1], flag])
        
    return pd.DataFrame(results, columns=["name", "code", "day", "30m", "Flag"])
            

# 机构调研股

In [6]:

names = """
    英科医疗 泉峰汽车 晓鸣股份 宝通科技 合兴股份 新五丰 新亚电子 东南网架 味知香 日发精机 楚天科技 拓维信息 汤姆猫 拓邦股份 
    华中数控 朗特智能 国林科技 科博达 瑞丰光电 润建股份 泰晶科技 聚飞光电 四川双马 国民技术 东方电热 云图控股 恒星科技 苏州固锝
    金杯电工 伊戈尔 湘潭电化 鑫铂股份 运达股份 东方电气 联创电子
"""

df = await check_rsi(names)
df

英科医疗	^^	[0.04, 0.64, 0.66]	日线
楚天科技	vv	[0.85, 0.88, 0.9]	日线
华中数控	v	[0.8, 0.93, 0.79]	30分钟
云海金属	^^	[0.09, 0.08, 0.11]	日线
泰晶科技	vv	[0.95, 0.91, 0.91]	日线
聚飞光电	v	[0.92, 0.92, 0.93]	30分钟
东方电热	v	[0.95, 0.98, 0.99]	30分钟
恒星科技	v	[0.9, 0.9, 0.9]	30分钟
苏州固锝	v	[0.97, 0.85, 0.74]	30分钟
金杯电工	v	[0.95, 0.95, 0.94]	30分钟
东方电气	v	[0.97, 0.83, 0.79]	30分钟
东方电气	vv	[0.68, 0.9, 0.91]	日线
联创电子	v	[0.9, 0.95, 0.86]	30分钟
联创电子	vv	[0.74, 0.75, 0.92]	日线


,name,code,day,30m,Flag
0,英科医疗,300677.XSHE,0.66,0.58,| ^^
1,宝通科技,300031.XSHE,0.70,0.63,
2,新五丰,600975.XSHG,0.70,0.78,
3,东南网架,002135.XSHE,0.78,0.76,
4,日发精机,002520.XSHE,0.50,0.86,
5,楚天科技,300358.XSHE,0.90,0.62,| vv
6,拓维信息,002261.XSHE,0.20,0.73,
7,汤姆猫,300459.XSHE,0.72,0.57,
8,拓邦股份,002139.XSHE,0.67,0.29,
9,华中数控,300161.XSHE,0.33,0.79,v


# 持仓股和指数

In [7]:
names = [    
    "中国西电", 
    "格林美", 
    "锦盛新材",
    "中国宝安",
    "惠泉啤酒",
    "方大集团",
    "天齐锂业",
    "晓鸣股份",
    "长城科技",
    "湖北宜化",
    "上证指数"
]

await check_rsi(names)

格林美	^^	[0.06, 0.06, 0.15]	日线
方大集团	v	[0.92, 0.87, 0.71]	30分钟
长城科技	^^	[0.17, 0.07, 0.08]	日线
上证指数	^^	[0.1, 0.09, 0.22]	日线


,name,code,day,30m,Flag
0,中国西电,601179.XSHG,0.16,0.85,
1,格林美,002340.XSHE,0.15,0.56,| ^^
2,中国宝安,000009.XSHE,0.46,0.71,
3,惠泉啤酒,600573.XSHG,0.80,0.72,
4,方大集团,000055.XSHE,0.83,0.71,v
5,天齐锂业,002466.XSHE,0.41,0.26,
6,长城科技,603897.XSHG,0.08,0.52,| ^^
7,湖北宜化,000422.XSHE,0.35,0.28,
8,上证指数,000001.XSHG,0.22,0.68,| ^^


In [24]:
m30("300967", 89.98)

0.92

In [20]:
day("605005", 7.63)

TypeError: type NoneType doesn't define __round__ method